In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import tensorflow as tf
import numpy as np

from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import concatenate

In [ ]:
import os
import numpy as np
import pandas as pd

import imageio

import matplotlib.pyplot as plt
%matplotlib inline

path = '/content/drive/MyDrive/Line_detection/dataset/non-aug/train/'
image_path = os.path.join(path, 'images/')
mask_path = os.path.join(path, 'masks/')
image_list = os.listdir(image_path)
mask_list = os.listdir(mask_path)
image_list = [image_path+i for i in image_list]
mask_list = [mask_path+i for i in mask_list]

valid_path = '/content/drive/MyDrive/Line_detection/dataset/non-aug/valid/'
valid_iamge_path = os.path.join(valid_path, 'images/')
valid_mask_path = os.path.join(valid_path, 'masks/')

In [ ]:
print(len(image_list), len(mask_list))
print(mask_list[0])
print(image_list[0])

print(len(valid_train_path), len(valid_mask_path))
print(valid_train_path[0])
print(valid_mask_path[0])

In [ ]:
import os
import numpy as np
import pandas as pd
import imageio
import matplotlib.pyplot as plt

path = '/content/drive/MyDrive/Line_detection/dataset/non-aug/train/'
image_path = os.path.join(path, 'images/')
mask_path = os.path.join(path, 'masks/')

image_list = os.listdir(image_path)
mask_list = os.listdir(mask_path)

image_list.sort()
mask_list.sort()


paired_images_masks = zip(image_list, mask_list)


for img_filename, mask_filename in paired_images_masks:
    img_filepath = os.path.join(image_path, img_filename)
    mask_filepath = os.path.join(mask_path, mask_filename)

    img = imageio.imread(img_filepath)
    mask = imageio.imread(mask_filepath)

    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.imshow(img)
    plt.title('Image')

    plt.subplot(1, 2, 2)
    plt.imshow(mask)
    plt.title('Mask')

    plt.tight_layout()
    plt.show()


In [ ]:
import os
import numpy as np
import pandas as pd
import imageio
import matplotlib.pyplot as plt

valid_path = '/content/drive/MyDrive/Line_detection/dataset/non-aug/valid/'
valid_image_path = os.path.join(path, 'images/')
valid_mask_path = os.path.join(path, 'masks/')

valid_image_list = os.listdir(image_path)
valid_mask_list = os.listdir(mask_path)

valid_image_list.sort()
valid_mask_list.sort()


paired_images_masks = zip(image_list, mask_list)


for img_filename, mask_filename in paired_images_masks:
    valid_img_filepath = os.path.join(valid_image_path, img_filename)
    valid_mask_filepath = os.path.join(valid_mask_path, mask_filename)

    img = imageio.imread(img_filepath)
    mask = imageio.imread(mask_filepath)

    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.imshow(img)
    plt.title('Image')

    plt.subplot(1, 2, 2)
    plt.imshow(mask)
    plt.title('Mask')

    plt.tight_layout()
    plt.show()


In [ ]:
N = 2
img = imageio.imread(image_list[N])
mask = imageio.imread(mask_list[N])
#mask = np.array([max(mask[i, j]) for i in range(mask.shape[0]) for j in range(mask.shape[1])]).reshape(img.shape[0], img.shape[1])

fig, arr = plt.subplots(1, 2, figsize=(14, 10))
arr[0].imshow(img)
arr[0].set_title('Image')
arr[1].imshow(mask[:, :, 0])
arr[1].set_title('Segmentation')

In [ ]:
image_list_ds = tf.data.Dataset.list_files(image_list, shuffle=False)
mask_list_ds = tf.data.Dataset.list_files(mask_list, shuffle=False)

for path in zip(image_list_ds.take(3), mask_list_ds.take(3)):
    print(path)

In [ ]:
image_filenames = tf.constant(image_list)
masks_filenames = tf.constant(mask_list)

dataset = tf.data.Dataset.from_tensor_slices((image_filenames, masks_filenames))

for image, mask in dataset.take(1):
    print(image)
    print(mask)

In [ ]:
myiter = iter(dataset)
next(myiter)[0]

In [ ]:
def process_path(image_path, mask_path):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_png(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)

    mask = tf.io.read_file(mask_path)
    mask = tf.image.decode_png(mask, channels=3)
    mask = tf.math.reduce_max(mask, axis=-1, keepdims=True)
    return img, mask

def preprocess(image, mask):
    input_image = tf.image.resize(image, (480, 480), method='nearest')
    input_mask = tf.image.resize(mask, (480, 480), method='nearest')
    # input_mask = input_mask.astype(tf.uint8)
    input_mask = tf.bitcast(tf.cast(input_mask/255, dtype=tf.int8), tf.uint8)
    return input_image, input_mask

image_ds = dataset.map(process_path)
processed_image_ds = image_ds.map(preprocess)

In [ ]:
import matplotlib.pyplot as plt

plt.subplot(1,2,1)
plt.imshow(next(iter(processed_image_ds))[0])
plt.subplot(1,2,2)
plt.imshow(next(iter(processed_image_ds))[1])

In [ ]:
img = next(iter(processed_image_ds))[1]
img = tf.reshape(img, (480,480)).numpy()
# img = img.astype(np.uint8)

plt.imshow(img)
plt.show()

from collections import Counter
Counter(img.flatten())

In [ ]:
img = next(iter(processed_image_ds))[0]
img

In [ ]:
def conv_block(inputs=None, n_filters=32, dropout_prob=0, max_pooling=True):


    conv = Conv2D(n_filters,
                  3,
                  activation='relu',
                  padding='same',
                  kernel_initializer='he_normal')(inputs)
    conv = Conv2D(n_filters,
                  3,
                  activation='relu',
                  padding='same',
                  kernel_initializer='he_normal')(conv)

    if dropout_prob > 0:
        conv = Dropout(rate=dropout_prob)(conv)


    if max_pooling:
        next_layer = MaxPooling2D()(conv)

    else:
        next_layer = conv

    skip_connection = conv

    return next_layer, skip_connection

In [ ]:
def upsampling_block(expansive_input, contractive_input, n_filters=32):

    up = Conv2DTranspose(
                 n_filters,
                 3,
                 strides=2,
                 padding='same')(expansive_input)


    merge = concatenate([up, contractive_input], axis=3)
    conv = Conv2D(n_filters,
                 3,
                 activation='relu',
                 padding='same',
                 kernel_initializer='he_normal')(merge)
    conv = Conv2D(n_filters,
                 3,
                 activation='relu',
                 padding='same',
                 kernel_initializer='he_normal')(conv)

    return conv

In [ ]:
def unet_model(input_size=(480, 480, 3), n_filters=32, n_classes=1):

    inputs = Input(input_size)
    cblock1 = conv_block(inputs, n_filters)
    cblock2 = conv_block(cblock1[0], n_filters * 2)
    cblock3 = conv_block(cblock2[0], n_filters * 4)
    cblock4 = conv_block(cblock3[0], n_filters * 8, dropout_prob=0.3)
    cblock5 = conv_block(cblock4[0], n_filters * 16, dropout_prob=0.3, max_pooling=False)
    ublock6 = upsampling_block(cblock5[0], cblock4[1], n_filters * 8)
    ublock7 = upsampling_block(ublock6, cblock3[1], n_filters * 4)
    ublock8 = upsampling_block(ublock7, cblock2[1], n_filters * 2)
    ublock9 = upsampling_block(ublock8, cblock1[1], n_filters)

    conv9 = Conv2D(n_filters,
                 3,
                 activation='relu',
                 padding='same',
                 kernel_initializer='he_normal')(ublock9)

    conv10 = Conv2D(n_classes, 1, activation='sigmoid', padding='same')(conv9)

    model = tf.keras.Model(inputs=inputs, outputs=conv10)

    return model

In [ ]:
img_height = 480
img_width = 480
num_channels = 3

unet = unet_model((img_height, img_width, num_channels))

In [ ]:
unet.summary()

In [ ]:
lr = 0.001
opt=tf.keras.optimizers.Adam(lr)

In [ ]:
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath="/content/drive/MyDrive/Line_detection/unet_model.h5",
    save_weights_only=True,
    monitor='loss',
    mode='min',
    save_best_only=True)

In [ ]:
unet.compile(optimizer=opt,
              loss=tf.keras.losses.BinaryCrossentropy(), #SparseCategoricalCrossentropy
              metrics=['accuracy'])

In [ ]:
def display(display_list):
    plt.figure(figsize=(15, 15))

    title = ['Input Image', 'True Mask', 'Predicted Mask']

    for i in range(len(display_list)):
        plt.subplot(1, len(display_list), i+1)
        plt.title(title[i])
        plt.imshow(tf.keras.preprocessing.image.array_to_img(display_list[i]))
        plt.axis('off')
    plt.show()

In [ ]:
for image, mask in image_ds.take(1):
    sample_image, sample_mask = image, mask
    print(mask.shape)
display([sample_image, sample_mask])

In [ ]:
for image, mask in processed_image_ds.take(1):
    sample_image, sample_mask = image, mask
    print(mask.shape)
display([sample_image, sample_mask])

In [ ]:
EPOCHS = 25
VAL_SUBSPLITS = 5
BUFFER_SIZE = 1
BATCH_SIZE = 4
train_dataset = processed_image_ds.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
print(processed_image_ds.element_spec)
model_history = unet.fit(train_dataset, epochs=EPOCHS, callbacks=[model_checkpoint_callback])

In [ ]:
unet.load_weights("/content/drive/MyDrive/Line_detection/unet_model.h5")

In [ ]:
img = next(iter(processed_image_ds))[0]
label = next(iter(processed_image_ds))[1]

plt.subplot(1,2,1)
plt.imshow(img)
plt.subplot(1,2,2)
plt.imshow(label)
plt.show()

In [ ]:
img = tf.expand_dims(img, axis=0)
prediction = unet.predict(img)
prediction.shape

In [ ]:
prediction

In [ ]:
plt.imshow((prediction.squeeze() > 0.1).astype(np.uint8), cmap = "gray")

In [ ]:
def create_mask(pred_mask):
    pred_mask = tf.argmax(pred_mask, axis=-1)
    pred_mask = pred_mask[..., tf.newaxis]
    return pred_mask[0]

In [ ]:
plt.plot(model_history.history["accuracy"])

In [ ]:
def show_predictions(dataset=None, num=1):
    """
    Displays the first image of each of the num batches
    """
    if dataset:
        for image, mask in dataset.take(num):
            pred_mask = unet.predict(image)
            display([image[0], mask[0], create_mask(pred_mask)])
    else:
        display([sample_image, sample_mask,
             create_mask(unet.predict(sample_image[tf.newaxis, ...]))])

In [ ]:
next(iter(train_dataset))[1][2].shape

In [ ]:
def new_display(dataset):

  for i in range(3):
    allImg = next(iter(dataset))
    img = allImg[0][0]
    label = allImg[1][0]
    prediction = tf.expand_dims(img, axis=0)
    prediction = unet.predict(prediction)
    prediction = prediction.squeeze()
    plt.subplot(3,3,1)
    plt.imshow(img)
    plt.subplot(3,3,2)
    plt.imshow(label)
    plt.subplot(3,3,3)
    plt.imshow(prediction)
    plt.show()

In [ ]:
import cv2
img = np.expand_dims(cv2.cvtColor(cv2.resize(cv2.imread("/content/a.png"), (480,480), interpolation = cv2.INTER_AREA), cv2.COLOR_BGR2RGB), axis = 0)
img.shape

In [ ]:
prediction = unet.predict(img).squeeze()
prediction = (prediction).astype(np.uint8)
plt.imshow((prediction).squeeze())

In [ ]:
prediction

In [ ]:
new_display(train_dataset)

In [ ]:
show_predictions(train_dataset, 6)